In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [2]:
MAIN_PATH = f'/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Nonos_synaptic_blockers'

In [3]:
filename = 'ID1851_N0_DIV100_DATE20240530_1342_spontaneous_.raw.h5'
PROCESSED_DATA_PATH = os.path.join(MAIN_PATH, f'Sorters/')
data = np.load(os.path.join(PROCESSED_DATA_PATH, f'{filename[:-3]}_processed.pkl'), allow_pickle=True)
spikes_extremum = pd.DataFrame(data['SPIKEMAT_EXTREMUM'])
spikes = np.array([[int(row[0]), float(row[1]), float(row[2])] for row in data['SPIKEMAT']])
PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')
OUTPUT_PATH = os.path.join(MAIN_PATH,f'Latency_plots/Latency_plots_TE_new')
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

SORTER_PATH = PROCESSED_DATA_PATH
FULL_SORTER_PATH = os.path.join(SORTER_PATH,'Sorter_'+filename)


area = filename.split('_')[1]
import pickle

# Open the file in binary mode
sys.modules['numpy.rec'] = np.rec
#with open(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), 'rb') as f:
#    data_te = np.array(pickle.load(f))
data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
exp_duration = data_te['EXPERIMENT_DURATION']
te_unit_pairs = pd.DataFrame(data_te['mTE'])
pairings = te_unit_pairs

with open(os.path.join(FULL_SORTER_PATH, 'wf_folder_curated/sparsity.json'), 'r') as file:
    sorting_info = json.load(file)

unit_ids = sorting_info['unit_ids']

spikes, electrodes_pre_all, electrodes_post_all, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all = get_electrode_unit_info_te(data, pairings, area, unit_ids)

In [4]:
def calculate_latency(lag, exp_duration, unit_to_el, input_ids, output_ids, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output=None):
    all_results = []
    
    for idx, input_id in enumerate(input_ids):
        latency_extremum = None
        validation = None
        
        for output_electrode in np.array(output_ids):
            latency_all = get_latency_all(spikes, spikes_extremum, unit_ids, unit_pre, unit_post, input_id, output_electrode)
            
            latency_all = latency_all[latency_all['latency'] < 10]
            input_before = latency_all[latency_all['category'] == 'input']
            output_before = latency_all[latency_all['category'] == 'output']
            
            if extremum_output is not None:
                latency_extremum = get_latency_with_extremum_V2(spikes_extremum, unit_ids, unit_pre, unit_post)
                latency_extremum = latency_extremum[latency_extremum['latency'] < 10]
                output_extremum = latency_extremum[latency_extremum['category'] == 'output']
                validation = check_pair(latency_extremum, lag, exp_duration)
                
                # Store results for this specific input_id and output_electrode
                all_results.append({
                    'input_id': input_id,
                    'output_electrode': output_electrode,
                    'latency_extremum': latency_extremum,
                    'validation': validation
                })
    
    return all_results

In [5]:
def run_for_all_files(filename,data_te,exp_duration,unit_ids,spikes,spikes_extremum, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all):
    all_validated_results = []

    for idx in range(len(pre_extremum_all)):
        pre_extremum = pre_extremum_all[idx]
        post_extremum = post_extremum_all[idx]
        unit_pre = unit_pre_all[idx]
        unit_post = unit_post_all[idx]
        lag = lag_all[idx]
        unit_to_el = data_te['UNIT_TO_EL']
        
        results = calculate_latency(lag, exp_duration, unit_to_el, pre_extremum, post_extremum, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output=True)
        
        for result in results:
            if result['validation'] == 'good':
                all_validated_results.append({
                    'source_electrode': pre_extremum[0],
                    'target_electrode': post_extremum[0],
                    'source_unit_id': unit_pre,
                    'target_unit_id': unit_post,
                    'lag': lag,
                    'latency_extremum': result['latency_extremum'],
                    'validation': result['validation']
                })

    # Save all the valid results in one pickle file
    if all_validated_results:
        with open(os.path.join(OUTPUT_PATH, f'{filename[:-3]}_latency.pkl'), 'wb') as f:
            pickle.dump(all_validated_results, f)

In [6]:
path = OUTPUT_PATH
saved_data = np.load(os.path.join(path, f'{filename[:-3]}_latency.pkl'), allow_pickle=True)

NameError: name 'OUTPUT_PATH' is not defined

In [16]:
saved_data

[{'source_electrode': 21820,
  'target_electrode': 14339,
  'source_unit_id': 0,
  'target_unit_id': 4,
  'lag': 1.0,
  'latency_extremum': array([( 0,   314.55, 0.  , 'input'), ( 1,  2280.6 , 0.  , 'input'),
         ( 2,  2622.45, 0.  , 'input'), ( 3,  4014.5 , 0.  , 'input'),
         ( 4,  4645.3 , 0.  , 'input'), ( 5,  5292.15, 0.  , 'input'),
         ( 6,  5422.15, 0.  , 'input'), ( 6,  5423.55, 1.4 , 'output'),
         ( 7,  7217.85, 0.  , 'input'), ( 8,  7899.95, 0.  , 'input'),
         ( 9,  7994.95, 0.  , 'input'), ( 9,  7996.35, 1.4 , 'output'),
         (10,  9072.2 , 0.  , 'input'), (11,  9415.7 , 0.  , 'input'),
         (12, 12642.2 , 0.  , 'input'), (13, 12819.5 , 0.  , 'input'),
         (14, 14891.4 , 0.  , 'input'), (15, 18529.85, 0.  , 'input'),
         (15, 18531.2 , 1.35, 'output'), (16, 19973.05, 0.  , 'input'),
         (16, 19974.35, 1.3 , 'output'), (17, 20152.  , 0.  , 'input'),
         (18, 21713.05, 0.  , 'input'), (19, 21714.55, 0.  , 'input'),
      

In [7]:
metadata = load_metadata_as_dataframe(file_path = os.path.join(MAIN_PATH, ''))

2025-03-18 22:35:09,397 Metadata successfully loaded. 
 First 5 rows: 
                                             Filename  DIV  Chip_ID  Network_ID
0  ID1297_N6_DIV14_DATE20241129_1356_spontaneous....   14     1297           6
1  ID1297_N3_DIV14_DATE20241129_1356_spontaneous....   14     1297           3
2  ID1297_N2_DIV14_DATE20241129_1356_spontaneous....   14     1297           2
3  ID1297_N5_DIV14_DATE20241129_1356_spontaneous....   14     1297           5
4  ID1297_N1_DIV14_DATE20241129_1356_spontaneous....   14     1297           1


In [8]:
PROCESSED_DATA_PATH = os.path.join(MAIN_PATH, f'Pickle_files/')
#filenames = [f for f in os.listdir(PROCESSED_DATA_PATH) if f.endswith('.pkl')]
a = 0
for filename in metadata['Filename']:
    chip_id = int(filename.split('_')[0].replace('ID', ''))
    div = int(filename.split('_')[2].replace('DIV', ''))
    try:
    #if a == 0:
        data = np.load(os.path.join(PROCESSED_DATA_PATH, f'{filename[:-3]}_processed.pkl'), allow_pickle=True)
        spikes_extremum = pd.DataFrame(data['SPIKEMAT_EXTREMUM'])
        spikes = np.array([[int(row[0]), float(row[1]), float(row[2])] for row in data['SPIKEMAT']])
        PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')
        OUTPUT_PATH = os.path.join(MAIN_PATH,f'Latency_plots/Latency_plots_TE_new')
        if not os.path.exists(OUTPUT_PATH):
            os.makedirs(OUTPUT_PATH)

        SORTER_PATH = f'/itet-stor/kvulic/neuronies/single_neurons/3_Student_Projects/Ferran/2_Processed_Data/{chip_id}_HeartShape/{chip_id}_DIV{div}_HeartShape/'
        FULL_SORTER_PATH = os.path.join(SORTER_PATH,'Sorter_'+filename)


        area = filename.split('_')[1]
        # Open the file in binary mode
        sys.modules['numpy.rec'] = np.rec
        #with open(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), 'rb') as f:
        #    data_te = np.array(pickle.load(f))
        data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
        exp_duration = data_te['EXPERIMENT_DURATION']
        te_unit_pairs = pd.DataFrame(data_te['mTE'])
        pairings = te_unit_pairs

        with open(os.path.join(FULL_SORTER_PATH, 'wf_folder_curated/sparsity.json'), 'r') as file:
            sorting_info = json.load(file)

        unit_ids = sorting_info['unit_ids']

        spikes, electrodes_pre_all, electrodes_post_all, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all = get_electrode_unit_info_te(data, pairings, area, unit_ids)
        run_for_all_files(filename,data_te,exp_duration,unit_ids,spikes,spikes_extremum, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all)
    except Exception as e:
        print(e)
        pass

[Errno 2] No such file or directory: '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Ferrans_processed_data_heart/biTE_new/ID1297_N6_DIV14_DATE20241129_1356_spontaneous.raw_processed_info_metrics.pkl'
[Errno 2] No such file or directory: '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Ferrans_processed_data_heart/biTE_new/ID1297_N3_DIV14_DATE20241129_1356_spontaneous.raw_processed_info_metrics.pkl'
[Errno 2] No such file or directory: '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Ferrans_processed_data_heart/biTE_new/ID1297_N2_DIV14_DATE20241129_1356_spontaneous.raw_processed_info_metrics.pkl'
[Errno 2] No such file or directory: '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Ferrans_processed_data_heart/biTE_new/ID1297_N5_DIV14_DATE20241129_1356_spontaneous.raw_processed_info_metrics.pkl'
[Errno 2] No such file o